In [1]:
import pandas as pd

from keras.src.utils import to_categorical

import numpy as np

from numpy.lib.stride_tricks import sliding_window_view


from sklearn.preprocessing import LabelEncoder

from keras.src.layers import LSTM
from keras import Sequential
from keras.src.layers import Embedding
from keras.src.layers import Dense

from sklearn.model_selection import train_test_split

from keras import Input


/Users/seb/git/master_thesis/nw_event_abstraction/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
train_r1 = pd.read_csv('../data/Train/R1/R1.csv')

In [3]:
# Mark start event of each BusinessActivity Instance
train_r1["activityStart"] = train_r1.groupby(["BusinessActivity","InstanceNumber",]).cumcount()==0
# Mark end event of each Business Activity Instance
train_r1["activityEnd"] = train_r1.groupby(["BusinessActivity","InstanceNumber",]).cumcount(ascending=False)==0
# Merge start and end columns to form labels
train_r1["task_position"] = train_r1.apply(lambda row: "position_start" if row["activityStart"] else ("position_end" if row["activityEnd"] else 'position_between'), axis=1)

train_r1 = train_r1.drop(["activityStart",'activityEnd'], axis=1)

In [4]:
# Label Encode the Message Type
messageTypeEncoder = LabelEncoder()
messageTypeEncoder.fit(train_r1["MessageType"].values)
# Shift the labels by 1 to exclude 0 as label (we use it for padding starting sequences)
train_r1["MessageType_labels"] = messageTypeEncoder.transform(train_r1["MessageType"]) + 1

# One hot encode the Labels
encoder = LabelEncoder()
y = encoder.fit_transform(train_r1["task_position"])
dummy_y = to_categorical(y)



# Todo build custom sliding window function forward + backward facing w. custom windowsize
sequence_length = 20
#labels = train_r1["MessageType_labels"].rolling(window=windowsize).agg(list)
sequences = sliding_window_view(train_r1["MessageType_labels"], sequence_length)

# pad first n-1 elements
pad_top = [[0] * (sequence_length - 1 - i) + sequences[0, 0:i + 1].tolist() for i in range(sequence_length - 1)]

X = np.insert(sequences,0, pad_top, axis=0).tolist()
X = np.asarray(X).astype(np.float32)

In [8]:
from keras.src.layers import LSTM
from keras import Input


def baseline_model():
    embedding_dim = 150

    vocab_size = len(messageTypeEncoder.classes_) + 1
    model = Sequential([
        Input(shape=(sequence_length,)),
        Embedding(input_dim=vocab_size, output_dim=embedding_dim),
        LSTM(units=64),
        Dense(3, activation='softmax')])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def stacked_LSTM(): 
    embedding_dim = 150
    vocab_size = len(messageTypeEncoder.classes_) +1
    model = Sequential([
        Input(shape= sequence_length,),
        Embedding(input_dim=vocab_size, output_dim=embedding_dim),
        LSTM(64, activation='relu', return_sequences=True),
        LSTM(64, activation='relu'),
        Dense(3, activation='softmax')])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['f1_score']) 
    return model

def stacked_LSTM_weighted():
    embedding_dim = 150
    vocab_size = len(messageTypeEncoder.classes_) +1
    model = Sequential([
        Input(shape= sequence_length,),
        Embedding(input_dim=vocab_size, output_dim=embedding_dim),
        LSTM(64, activation='relu', return_sequences=True),
        LSTM(64, activation='relu'),
        Dense(3, activation='softmax')])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['f1_score']) 
    return model


    



In [10]:
x_train,x_test, y_train, y_test = train_test_split(X, dummy_y, test_size=0.2, shuffle=True, random_state=42)

In [12]:
from scikeras.wrappers import KerasClassifier

estimator = KerasClassifier(model=baseline_model, epochs=5, batch_size=1000, verbose=1)

In [13]:

estimator.fit(x_train, y_train)
pred = estimator.predict(x_test)

2023-12-01 14:26:39.220819: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2023-12-01 14:26:39.220877: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2023-12-01 14:26:39.220887: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2023-12-01 14:26:39.221150: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-12-01 14:26:39.221546: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/5


2023-12-01 14:26:40.749515: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


1285/1285 ━━━━━━━━━━━━━━━━━━━━ 700s 542ms/step - accuracy: 0.9945 - loss: 0.0254
Epoch 2/5
1137/1285 ━━━━━━━━━━━━━━━━━━━━ 1:17 526ms/step - accuracy: 0.9988 - loss: 0.0035


KeyboardInterrupt

